# Data Cleaning
### This page will divide data cleaning into two parts. 
### The first part is the Python method. Here we focus on the data set of Twitter. It will include the data cleaning process of texts including keyword," work, job"  from Twitter,the data cleaning process of texts including keyword," job hunting"  from Twitter,the data cleaning process of texts including keyword," new graduate"  from Twitter,the data cleaning process of texts including keyword," retire"  from Twitter and two label, tweeted, so see if the text of a tweet is widely recognized by people or not, and favorited, so see if the text tweet is widely aggreed by people or not. If you want to get the data, you can come back to "Home" click the "Data" tab in the menu, and if you want to get the code, you can just copy it from below, or you can get it by clicking the "Code" tab in the menu. 
### In general, I get the all needed columns and then clean the text data by using Spacy pipeline for text normalization and preprocessing. I use "re" package to identity the formats of most URLs,  emails. And in this part, numbers in the text is not important for my topic so I replace them with "numbr" to explore the relationship between number and retweeted. Then, I remove commas in numbers, punctuation and other junk. After these process, I get a much clean dataset. Then I use CountVectorizer, TfidfVectorizer to get BOW features for further exploration.
### The label "retweeted" is very strightfoward, which is designed as when retweet_count is larger than 0,  then retweeted is True, else False. 

### More:  make attention about that I didn't save the "X_bow" in this part, you can use the code at the last part in python part to get it, you can find "textcleaning_py1.csv","textcleaning_py2.csv","textcleaning_py3.csv","textcleaning_py4.csv", which contains "id", "label", "text".




The first dataset we need to deal with is "raw_data1.csv", that is, the twitter texts with the keyword of "new graduate".

In [45]:
import pandas as pd
r = pd.read_csv("raw_data1.csv")

### Select all needed columns

In [46]:
r.head()

,Unnamed: 0,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,lang,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,extended_entities,withheld_in_countries
0,0,Sun Dec 04 04:49:26 +0000 2022,1599264591531638784,1599264591531638784,RT @scjchurch_en: New York Feature\nZion Chris...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Dec 04 04:48:35 +0000 2022,1599264378875846656,1599264378875846656,RT @ARYNEWSOFFICIAL: MBBS Graduate sets new wo...,False,"{'hashtags': [{'text': 'ARYNews', 'indices': [...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,False,NaN,NaN,NaN,NaN,NaN
2,2,Sun Dec 04 04:41:52 +0000 2022,1599262688466477056,1599262688466477056,RT @TEDchris: 5/6 It really feels like the wor...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Sun Dec 04 04:35:54 +0000 2022,1599261187635830785,1599261187635830785,RT @NamyrMya: Wait if they're gonna announce n...,False,{'hashtags': [{'text': 'BeOnCloudNewYearParty'...,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Sun Dec 04 04:32:15 +0000 2022,1599260268042760194,1599260268042760194,RT @ConceptTVNews: MBBS graduate Hafiz Muhamma...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
r1 = r[["id","text","truncated","entities","metadata","user","is_quote_status","favorite_count","retweet_count","favorited","retweeted","lang","possibly_sensitive"]]

### Deal with Text data
#### use Spacy pipeline for text normalization & preprocessing

### This is a quick look of text data

In [48]:
t = r1[["id","text"]]

In [49]:
t.head()

,id,text
0,1599264591531638784,RT @scjchurch_en: New York Feature\nZion Chris...
1,1599264378875846656,RT @ARYNEWSOFFICIAL: MBBS Graduate sets new wo...
2,1599262688466477056,RT @TEDchris: 5/6 It really feels like the wor...
3,1599261187635830785,RT @NamyrMya: Wait if they're gonna announce n...
4,1599260268042760194,RT @ConceptTVNews: MBBS graduate Hafiz Muhamma...


### We can find that the text is mixed with name, numbers, uppercase, which we should deal with it, so I use a pipeline to match URLs, emails, numbers and punctuation.

In [50]:
import re
import spacy
from spacy.language import Language

pipeline = spacy.load('en_core_web_sm')

# http://emailregex.com/
email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

# replace = [ (pattern-to-replace, replacement),  ...]
replace = [
    (r"<a[^>]*>(.*?)</a>", r"\1"),  # Matches most URLs
    (email_re, "email"),            # Matches emails
    (r"(?<=\d),(?=\d)", ""),        # Remove commas in numbers
    (r"\d+", "number"),              # Map digits to special token <numbr>
    (r"[\t\n\r\*\.\@\,\-\/]", " "), # Punctuation and other junk
    (r"\s+", " ")                   # Stips extra whitespace
]
train_sentences = []
for i, d in enumerate(t['text']):
    for repl in replace:
        d = re.sub(repl[0], repl[1], d)
    train_sentences.append(d)


@Language.component("anly501project")
def ng20_preprocess(doc):
    tokens = [token for token in doc 
              if not any((token.is_stop, token.is_punct))]
    tokens = [token.lemma_.lower().strip() for token in tokens]
    tokens = [token for token in tokens if token]
    return " ".join(tokens)


pipeline.add_pipe("anly501project")

<function __main__.ng20_preprocess(doc)>

### Pass data through our Spacy pipeline

In [51]:
texts = []
for sent in train_sentences:
    texts.append(pipeline(sent))

In [52]:
texts[0:5]

['rt scjchurch_en new york feature zion christian mission center break record high graduate world christian insti',
 'rt arynewsofficial mbbs graduate set new world record win number gold medal read https t co nbnumberyasdvznumber arynew https t',
 'rt tedchris number number feel like world change writer editor designer coder artist suspect countless',
 'rt namyrmya wait go to announce new actor s beoncloudnewyearparty amp previously search number amp',
 'rt concepttvnew mbbs graduate hafiz muhammad waleed malik ameeruddin medical college lahore set new record win number gold']

### Compute number of unique words (vocabulary size)

In [53]:
vocab_size = len(set(" ".join(texts).split(" ")))
vocab_size

8438

### Build the term-document matrix, which is BOW features in this case.

In [54]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
bow_featurizer = CountVectorizer(max_features=vocab_size, max_df=0.95, min_df=0.005, stop_words='english')
tfidf_featurizer = TfidfVectorizer(max_features=vocab_size, max_df=0.95, stop_words='english')
X_bow = bow_featurizer.fit_transform(texts)
X_tfidf = tfidf_featurizer.fit_transform(texts)
type(X_bow), X_bow.shape

(scipy.sparse._csr.csr_matrix, (6000, 329))

In [55]:
type(X_tfidf),X_tfidf.shape

(scipy.sparse._csr.csr_matrix, (6000, 8201))

### Create a index-to-word map

In [56]:
idxtoword = {idx: word for word, idx in bow_featurizer.vocabulary_.items()}

### Create a label

In [57]:
label1 = r1[["id","retweet_count"]]
label2 = r1[["id","favorite_count"]]

#### This label, retweeted means if this tweet is retweeted by other people.
#### This label, favorited means if this tweet is favorited by other people.

In [58]:
label1['retweeted'] = label1['retweet_count'].map(lambda c:True if c>0 else False)
label2['favorited'] = label2['favorite_count'].map(lambda c:True if c>0 else False)

/var/folders/_0/b954bf3s3y76xgcc4xp4kyhc0000gn/T/ipykernel_15129/3589697755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label1['retweeted'] = label1['retweet_count'].map(lambda c:True if c>0 else False)
/var/folders/_0/b954bf3s3y76xgcc4xp4kyhc0000gn/T/ipykernel_15129/3589697755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label2['favorited'] = label2['favorite_count'].map(lambda c:True if c>0 else False)


In [59]:
label1.head()

,id,retweet_count,retweeted
0,1599264591531638784,17,True
1,1599264378875846656,112,True
2,1599262688466477056,15,True
3,1599261187635830785,1,True
4,1599260268042760194,5,True


In [60]:
label2.head()

,id,favorite_count,favorited
0,1599264591531638784,0,False
1,1599264378875846656,0,False
2,1599262688466477056,0,False
3,1599261187635830785,0,False
4,1599260268042760194,0,False


In [61]:
tt = pd.DataFrame(texts,columns =['Clean_Text'])

### As we can see, the text data is much cleaner as a lot of URLs are changed to "https" and all the words are in lowercase. 

In [62]:
tt = pd.concat([label1,tt],axis = 1)
tt = pd.concat([label2,tt],axis = 1)

In [63]:
tt.to_csv("textcleaning_py1.csv",index = 0)

## For the remaining three data sets, we adopt the same processing method.

The second dataset we need to deal with is "raw_data2.csv", that is, the twitter texts with the keyword of "job hunting".

In [64]:
import pandas as pd
r = pd.read_csv("raw_data2.csv")

### Select all needed columns

In [65]:
r

,Unnamed: 0,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,lang,retweeted_status,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status
0,0,Tue Dec 06 16:28:51 +0000 2022,1600165384362938368,1600165384362938368,Anyone got experience interview for job in you...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Tue Dec 06 16:28:28 +0000 2022,1600165286090461192,1600165286090461192,Nothing kills your self-esteem more than job h...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Tue Dec 06 16:27:55 +0000 2022,1600165148844445700,1600165148844445700,RT @shes_thembeka: The most annoying part abou...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,{'created_at': 'Tue Dec 06 16:12:17 +0000 2022...,NaN,NaN,NaN,NaN,NaN
3,3,Tue Dec 06 16:25:41 +0000 2022,1600164584148860928,1600164584148860928,I got 2 more exams and a presentation this wee...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Tue Dec 06 16:24:59 +0000 2022,1600164408616898562,1600164408616898562,#jobsearch #jobopenings Email Job-Hunting Tips...,False,"{'hashtags': [{'text': 'jobsearch', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,0,False,False,en,NaN,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6047,6047,Tue Nov 29 10:51:52 +0000 2022,1597543863102865410,1597543863102865410,RT @Notyatyi: Job hunting /unemployment.,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,{'created_at': 'Tue Nov 29 05:17:01 +0000 2022...,NaN,NaN,1.597267e+18,1.597267e+18,NaN
6048,6048,Tue Nov 29 10:48:42 +0000 2022,1597543066256015360,1597543066256015360,If you are hunting for a DevOps engineering jo...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
6049,6049,Tue Nov 29 10:47:54 +0000 2022,1597542864908877824,1597542864908877824,RT @Notyatyi: Job hunting /unemployment.,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,{'created_at': 'Tue Nov 29 05:17:01 +0000 2022...,NaN,NaN,1.597267e+18,1.597267e+18,NaN
6050,6050,Tue Nov 29 10:44:44 +0000 2022,1597542068246544384,1597542068246544384,RT @Notyatyi: Job hunting /unemployment.,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,{'created_at': 'Tue Nov 29 05:17:01 +0000 2022...,NaN,NaN,1.597267e+18,1.597267e+18,NaN


In [66]:
r1 = r[["id","text","truncated","entities","metadata","user","is_quote_status","favorite_count","retweet_count","favorited","retweeted","lang","possibly_sensitive"]]

### Deal with Text data
#### use Spacy pipeline for text normalization & preprocessing

### This is a quick look of text data

In [67]:
t = r1[["id","text"]]

In [68]:
t.head()

,id,text
0,1600165384362938368,Anyone got experience interview for job in you...
1,1600165286090461192,Nothing kills your self-esteem more than job h...
2,1600165148844445700,RT @shes_thembeka: The most annoying part abou...
3,1600164584148860928,I got 2 more exams and a presentation this wee...
4,1600164408616898562,#jobsearch #jobopenings Email Job-Hunting Tips...


### We can find that the text is mixed with name, numbers, uppercase, which we should deal with it, so I use a pipeline to match URLs, emails, numbers and punctuation.

In [69]:
texts = []
for sent in train_sentences:
    texts.append(pipeline(sent))

In [70]:
texts[0:5]

['rt scjchurch_en new york feature zion christian mission center break record high graduate world christian insti',
 'rt arynewsofficial mbbs graduate set new world record win number gold medal read https t co nbnumberyasdvznumber arynew https t',
 'rt tedchris number number feel like world change writer editor designer coder artist suspect countless',
 'rt namyrmya wait go to announce new actor s beoncloudnewyearparty amp previously search number amp',
 'rt concepttvnew mbbs graduate hafiz muhammad waleed malik ameeruddin medical college lahore set new record win number gold']

### Compute number of unique words (vocabulary size)

In [71]:
vocab_size = len(set(" ".join(texts).split(" ")))
vocab_size

8438

### Build the term-document matrix, which is BOW features in this case.

In [72]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
bow_featurizer = CountVectorizer(max_features=vocab_size, max_df=0.95, min_df=0.005, stop_words='english')
tfidf_featurizer = TfidfVectorizer(max_features=vocab_size, max_df=0.95, stop_words='english')
X_bow = bow_featurizer.fit_transform(texts)
X_tfidf = tfidf_featurizer.fit_transform(texts)
type(X_bow), X_bow.shape

(scipy.sparse._csr.csr_matrix, (6000, 329))

In [73]:
type(X_tfidf),X_tfidf.shape

(scipy.sparse._csr.csr_matrix, (6000, 8201))

### Create a index-to-word map

In [74]:
idxtoword = {idx: word for word, idx in bow_featurizer.vocabulary_.items()}

### Create a label

In [75]:
label1 = r1[["id","retweet_count"]]
label2 = r1[["id","favorite_count"]]

#### This label, retweeted means if this tweet is retweeted by other people.
#### This label, favorited means if this tweet is favorited by other people.

In [76]:
label1['retweeted'] = label1['retweet_count'].map(lambda c:True if c>0 else False)
label2['favorited'] = label2['favorite_count'].map(lambda c:True if c>0 else False)

/var/folders/_0/b954bf3s3y76xgcc4xp4kyhc0000gn/T/ipykernel_15129/3589697755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label1['retweeted'] = label1['retweet_count'].map(lambda c:True if c>0 else False)
/var/folders/_0/b954bf3s3y76xgcc4xp4kyhc0000gn/T/ipykernel_15129/3589697755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label2['favorited'] = label2['favorite_count'].map(lambda c:True if c>0 else False)


In [77]:
label1.head()

,id,retweet_count,retweeted
0,1600165384362938368,0,False
1,1600165286090461192,0,False
2,1600165148844445700,1,True
3,1600164584148860928,0,False
4,1600164408616898562,0,False


In [78]:
label2.head()

,id,favorite_count,favorited
0,1600165384362938368,0,False
1,1600165286090461192,0,False
2,1600165148844445700,0,False
3,1600164584148860928,0,False
4,1600164408616898562,0,False


### As we can see, the text data is much cleaner as a lot of URLs are changed to "https" and all the words are in lowercase. 

In [79]:
tt = pd.concat([label1,tt],axis = 1)
tt = pd.concat([label2,tt],axis = 1)

In [80]:
tt.to_csv("textcleaning_py2.csv",index = 0)

The third dataset we need to deal with is "raw_data3.csv", that is, the twitter texts with the keyword of "job or work".

In [81]:
import pandas as pd
r = pd.read_csv("raw_data3.csv")

### Select all needed columns

In [82]:
r.head()

,Unnamed: 0,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,...,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status
0,0,Tue Dec 06 16:32:45 +0000 2022,1600166363456507904,1600166363456507904,RT @CNNHeroes: This is your last chance to vot...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/#!/download/ipad"" ...",NaN,...,1,0,False,False,en,NaN,NaN,NaN,NaN,NaN
1,1,Tue Dec 06 16:32:45 +0000 2022,1600166363347419136,1600166363347419136,work pays off. 1 good life meet slomy A0.0Z...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,0,False,False,en,"{'media': [{'id': 1600135851400638464, 'id_str...",False,NaN,NaN,NaN
2,2,Tue Dec 06 16:32:45 +0000 2022,1600166362420170762,1600166362420170762,RT @CSSZNi: Pogba in the 2018 World Cup has to...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,19,0,False,False,en,NaN,NaN,NaN,NaN,NaN
3,3,Tue Dec 06 16:32:45 +0000 2022,1600166362139480066,1600166362139480066,Most urban corporation are controlled by BJP. ...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,0,False,False,en,NaN,False,1.600165e+18,1.600165e+18,{'created_at': 'Tue Dec 06 16:25:42 +0000 2022...
4,4,Tue Dec 06 16:32:44 +0000 2022,1600166361803653131,1600166361803653131,RT @KTP92: “World powers fear a united people ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://tweetoshi.com"" rel=""nofollow""...",NaN,...,38,0,False,False,en,NaN,NaN,NaN,NaN,NaN


In [83]:
r1 = r[["id","text","truncated","entities","metadata","user","is_quote_status","favorite_count","retweet_count","favorited","retweeted","lang","possibly_sensitive"]]

### Deal with Text data
#### use Spacy pipeline for text normalization & preprocessing

### This is a quick look of text data

In [84]:
t = r1[["id","text"]]

In [85]:
t.head()

,id,text
0,1600166363456507904,RT @CNNHeroes: This is your last chance to vot...
1,1600166363347419136,work pays off. 1 good life meet slomy A0.0Z...
2,1600166362420170762,RT @CSSZNi: Pogba in the 2018 World Cup has to...
3,1600166362139480066,Most urban corporation are controlled by BJP. ...
4,1600166361803653131,RT @KTP92: “World powers fear a united people ...


### We can find that the text is mixed with name, numbers, uppercase, which we should deal with it, so I use a pipeline to match URLs, emails, numbers and punctuation.

### Pass data through our Spacy pipeline

In [86]:
texts = []
for sent in train_sentences:
    texts.append(pipeline(sent))

In [87]:
texts[0:5]

['rt scjchurch_en new york feature zion christian mission center break record high graduate world christian insti',
 'rt arynewsofficial mbbs graduate set new world record win number gold medal read https t co nbnumberyasdvznumber arynew https t',
 'rt tedchris number number feel like world change writer editor designer coder artist suspect countless',
 'rt namyrmya wait go to announce new actor s beoncloudnewyearparty amp previously search number amp',
 'rt concepttvnew mbbs graduate hafiz muhammad waleed malik ameeruddin medical college lahore set new record win number gold']

### Compute number of unique words (vocabulary size)

In [88]:
vocab_size = len(set(" ".join(texts).split(" ")))
vocab_size

8438

### Build the term-document matrix, which is BOW features in this case.

In [89]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
bow_featurizer = CountVectorizer(max_features=vocab_size, max_df=0.95, min_df=0.005, stop_words='english')
tfidf_featurizer = TfidfVectorizer(max_features=vocab_size, max_df=0.95, stop_words='english')
X_bow = bow_featurizer.fit_transform(texts)
X_tfidf = tfidf_featurizer.fit_transform(texts)
type(X_bow), X_bow.shape

(scipy.sparse._csr.csr_matrix, (6000, 329))

In [90]:
type(X_tfidf),X_tfidf.shape

(scipy.sparse._csr.csr_matrix, (6000, 8201))

### Create a index-to-word map

In [91]:
idxtoword = {idx: word for word, idx in bow_featurizer.vocabulary_.items()}

### Create a label

In [92]:
label1 = r1[["id","retweet_count"]]
label2 = r1[["id","favorite_count"]]

#### This label, retweeted means if this tweet is retweeted by other people.
#### This label, favorited means if this tweet is favorited by other people.

In [93]:
label1['retweeted'] = label1['retweet_count'].map(lambda c:True if c>0 else False)
label2['favorited'] = label2['favorite_count'].map(lambda c:True if c>0 else False)

/var/folders/_0/b954bf3s3y76xgcc4xp4kyhc0000gn/T/ipykernel_15129/3589697755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label1['retweeted'] = label1['retweet_count'].map(lambda c:True if c>0 else False)
/var/folders/_0/b954bf3s3y76xgcc4xp4kyhc0000gn/T/ipykernel_15129/3589697755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label2['favorited'] = label2['favorite_count'].map(lambda c:True if c>0 else False)


In [94]:
label1.head()

,id,retweet_count,retweeted
0,1600166363456507904,1,True
1,1600166363347419136,0,False
2,1600166362420170762,19,True
3,1600166362139480066,0,False
4,1600166361803653131,38,True


In [95]:
label2.head()

,id,favorite_count,favorited
0,1600166363456507904,0,False
1,1600166363347419136,0,False
2,1600166362420170762,0,False
3,1600166362139480066,0,False
4,1600166361803653131,0,False


In [96]:
tt = pd.DataFrame(texts,columns =['Clean_Text'])

### As we can see, the text data is much cleaner as a lot of URLs are changed to "https" and all the words are in lowercase. 

In [97]:
tt = pd.concat([label1,tt],axis = 1)
tt = pd.concat([label2,tt],axis = 1)

In [98]:
tt.to_csv("textcleaning_py3.csv",index = 0)

The second dataset we need to deal with is "raw_data4.csv", that is, the twitter texts with the keyword of "retire".

In [99]:
import pandas as pd
r = pd.read_csv("raw_data4.csv")

### Select all needed columns

In [100]:
r.head()

,Unnamed: 0,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,lang,quoted_status_id,quoted_status_id_str,quoted_status,possibly_sensitive,extended_entities,withheld_in_countries
0,0,Tue Dec 06 16:46:49 +0000 2022,1600169903952388097,1600169903952388097,RT @bcliberals: According to BC Family Doctors...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Tue Dec 06 16:46:46 +0000 2022,1600169889607450629,1600169889607450629,RT @DeborahDian: Where should you retire? Wha...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/#!/download/ipad"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Tue Dec 06 16:46:28 +0000 2022,1600169817964617728,1600169817964617728,RT @roadto_billions: My plan is to retire from...,False,"{'hashtags': [{'text': 'dividends', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Tue Dec 06 16:46:20 +0000 2022,1600169780840759299,1600169780840759299,Crazy they let this man “retire” when it’s cau...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,1.599923e+18,1.599923e+18,{'created_at': 'Tue Dec 06 00:24:56 +0000 2022...,False,NaN,NaN
4,4,Tue Dec 06 16:46:19 +0000 2022,1600169779519574016,1600169779519574016,All the tweets about #tombrady should retire a...,True,"{'hashtags': [{'text': 'tombrady', 'indices': ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
r1 = r[["id","text","truncated","entities","metadata","user","is_quote_status","favorite_count","retweet_count","favorited","retweeted","lang","possibly_sensitive"]]

### Deal with Text data
#### use Spacy pipeline for text normalization & preprocessing

### This is a quick look of text data

In [102]:
t = r1[["id","text"]]

In [103]:
t.head()

,id,text
0,1600169903952388097,RT @bcliberals: According to BC Family Doctors...
1,1600169889607450629,RT @DeborahDian: Where should you retire? Wha...
2,1600169817964617728,RT @roadto_billions: My plan is to retire from...
3,1600169780840759299,Crazy they let this man “retire” when it’s cau...
4,1600169779519574016,All the tweets about #tombrady should retire a...


### We can find that the text is mixed with name, numbers, uppercase, which we should deal with it, so I use a pipeline to match URLs, emails, numbers and punctuation.

In [104]:
texts = []
for sent in train_sentences:
    texts.append(pipeline(sent))

In [105]:
texts[0:5]

['rt scjchurch_en new york feature zion christian mission center break record high graduate world christian insti',
 'rt arynewsofficial mbbs graduate set new world record win number gold medal read https t co nbnumberyasdvznumber arynew https t',
 'rt tedchris number number feel like world change writer editor designer coder artist suspect countless',
 'rt namyrmya wait go to announce new actor s beoncloudnewyearparty amp previously search number amp',
 'rt concepttvnew mbbs graduate hafiz muhammad waleed malik ameeruddin medical college lahore set new record win number gold']

### Compute number of unique words (vocabulary size)

In [106]:
vocab_size = len(set(" ".join(texts).split(" ")))
vocab_size

8438

### Build the term-document matrix, which is BOW features in this case.

In [107]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
bow_featurizer = CountVectorizer(max_features=vocab_size, max_df=0.95, min_df=0.005, stop_words='english')
tfidf_featurizer = TfidfVectorizer(max_features=vocab_size, max_df=0.95, stop_words='english')
X_bow = bow_featurizer.fit_transform(texts)
X_tfidf = tfidf_featurizer.fit_transform(texts)
type(X_bow), X_bow.shape

(scipy.sparse._csr.csr_matrix, (6000, 329))

In [108]:
type(X_tfidf),X_tfidf.shape

(scipy.sparse._csr.csr_matrix, (6000, 8201))

### Create a index-to-word map

In [109]:
idxtoword = {idx: word for word, idx in bow_featurizer.vocabulary_.items()}

### Create a label

In [110]:
label1 = r1[["id","retweet_count"]]
label2 = r1[["id","favorite_count"]]

#### This label, retweeted means if this tweet is retweeted by other people.
#### This label, favorited means if this tweet is favorited by other people.

In [111]:
label1['retweeted'] = label1['retweet_count'].map(lambda c:True if c>0 else False)
label2['favorited'] = label2['favorite_count'].map(lambda c:True if c>0 else False)

/var/folders/_0/b954bf3s3y76xgcc4xp4kyhc0000gn/T/ipykernel_15129/3589697755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label1['retweeted'] = label1['retweet_count'].map(lambda c:True if c>0 else False)
/var/folders/_0/b954bf3s3y76xgcc4xp4kyhc0000gn/T/ipykernel_15129/3589697755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label2['favorited'] = label2['favorite_count'].map(lambda c:True if c>0 else False)


In [112]:
label1.head()

,id,retweet_count,retweeted
0,1600169903952388097,9,True
1,1600169889607450629,1,True
2,1600169817964617728,1,True
3,1600169780840759299,0,False
4,1600169779519574016,0,False


In [113]:
label2.head()

,id,favorite_count,favorited
0,1600169903952388097,0,False
1,1600169889607450629,0,False
2,1600169817964617728,0,False
3,1600169780840759299,0,False
4,1600169779519574016,0,False


### As we can see, the text data is much cleaner as a lot of URLs are changed to "https" and all the words are in lowercase. 

In [114]:
tt = pd.concat([label1,tt],axis = 1)
tt = pd.concat([label2,tt],axis = 1)

In [115]:
tt.to_csv("textcleaning_py4.csv",index = 0)